In [2]:
import folium
import geopandas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as spst
import pandas_profiling

import warnings
warnings.simplefilter(action='ignore')
%config InlineBackend.figure_format = 'retina'
pd.set_option('display.max_columns', None)

plt.rc('font', family='AppleGothic')
sns.set(font="AppleGothic",   
        rc={"axes.unicode_minus":False},    # 마이너스 부호 깨짐 현상 해결
        style='whitegrid')

In [3]:
df_dong1 = pd.read_csv('행정동별_지하철_인구이동_사업체_버스_상권.csv', encoding='cp949')
df_gu1 = pd.read_csv('자치구별_주민등록_예산_인구이동_사업체_상권.csv', encoding='cp949')

df_gu2 = pd.read_csv('(구별 통합) 자동차, 주차장, 불법주차.csv', encoding='utf-8')
df_dong2 = pd.read_csv('(행정동별 통합) 자동차, 주차장, 불법주차.csv', encoding='utf-8')

# 필요한 열만
df_dong2 = df_dong2[['행정동명', '자동차등록대수', '공영 면수', '공영 개수', '민영 개수', '민영 면수', '주차장 면수', '단속건수']]
df_gu2 = df_gu2[['자치구', '2022 단속건수']]
df_dong2.columns = ['행정동', '자동차등록대수', '공영 면수', '공영 개수', '민영 개수', '민영 면수', '주차장 면수', '단속건수']

# merge 
df_gu = pd.merge(df_gu1, df_gu2, on='자치구', how='outer')
df_dong = pd.merge(df_dong1, df_dong2, on='행정동', how='outer')

# 거의 0 값을 가지는 광업 제거
df_dong.drop(['광업'], axis=1, inplace=True)

In [4]:
df_gu.columns

Index(['자치구', '총 세대수', '총 인구수', '세대당인구', '총 예산', '주차장사업비', '순이동', '총상주인구(명)',
       '총유입인구(명)', '통근_유입인구(명)', '통학_유입인구(명)', '총유출인구(명)', '통근_유출인구(명)',
       '통학_유출인구(명)', '총주간인구(명)', '주간인구지수(%)', '택시운송업', '한식_일반_음식점업',
       '용달_화물자동차_운송업', '부동산_중개_및_대리업', '두발미용업', '커피전문점', '기타_주점업',
       '남녀용_겉옷_및_셔츠_도매업', '일반_교과_학원', '한식_육류요리_전문점', '총상권개수', '2022 단속건수'],
      dtype='object')

In [5]:
df_dong.columns

Index(['행정동', '승차총승객수', '하차총승객수', '순이동', '사업체수', '종사자수', '농업 임업 및 어업', '제조업',
       '전기 가스 증기 및 공기조절 공급업', '수도 하수 및 폐기물 처리 원료 재생업', '건설업', '도매 및 소매업',
       '운수 및 창고업', '숙박 및 음식점업', '정보통신업', '금융 및 보험업', '부동산업', '전문 과학 및 기술 서비스업',
       '사업시설 관리 사업 지원 및 임대 서비스업', '공공행정 국방 및 사회보장 행정', '교육 서비스업',
       '보건업 및 사회복지 서비스업', '예술 스포츠 및 여가관련 서비스업', '협회 및 단체 수리 및 기타 개인 서비스업',
       '택시운송업', '버스승차총승객수', '버스하차총승객수', '총상권개수', '총주택수', '총단독주택', '일반단독주택',
       '다가구주택', '영업겸용주택', '아파트', '연립주택', '다세대주택', '비거주용건물내주택', '자동차등록대수',
       '공영 면수', '공영 개수', '민영 개수', '민영 면수', '주차장 면수', '단속건수'],
      dtype='object')

In [6]:
df_gu.to_csv('df_gu_final.csv', index=False)

In [7]:
df_dong.to_csv('df_dong_final.csv', index=False)

## 자치구별

In [73]:
# 서울시 자치구 경계 데이터 불러오기

jpath = '자치구.json'
seoul_geo = json.load(open(jpath, encoding='utf-8'))

In [81]:
g_map = folium.Map(location = [37.5642135, 127.0016985],
                   tiles='CartoDB positron',
                   zoom_start = 11.2)
fmap = folium.Choropleth(geo_data = seoul_geo,
                        data=df_gu,
                        columns=['자치구', '2022 단속건수'],
                        key_on = 'feature.properties.sggnm', 
                        fill_color='PuRd',    #YlOrRd
                        fill_opacity=0.7,
                        line_opacity=0.3, 
                        legend_name='단속건수', 
                        highlight=False).add_to(g_map)
folium.LayerControl().add_to(g_map)

fmap.geojson.zoom_on_click = False
fmap.geojson.add_child( folium.features.GeoJsonTooltip(['sggnm'], lables=False))
g_map.save('구별_단속건수.html')

In [84]:
g_map = folium.Map(location = [37.5642135, 127.0016985],
                   tiles='CartoDB positron',
                   zoom_start = 11.2)
fmap = folium.Choropleth(geo_data = seoul_geo,
                        data=df_gu,
                        columns=['자치구', '주차장사업비'],
                        key_on = 'feature.properties.sggnm', 
                        fill_color='PuRd',    #YlOrRd
                        fill_opacity=0.7,
                        line_opacity=0.3, 
                        legend_name='단속건수', 
                        highlight=False).add_to(g_map)
folium.LayerControl().add_to(g_map)

fmap.geojson.zoom_on_click = False
fmap.geojson.add_child( folium.features.GeoJsonTooltip(['sggnm'], lables=False))
g_map.save('구별_주차장사업비.html')

## 행정동 앞에 구 붙여주기

In [8]:
for i in range(len(df_dong)):
    df_dong.iloc[i, 0] = '강서구 ' + df_dong.iloc[i, 0]

In [14]:
# 강서구 행정동 경계 데이터 불러오기

jpath = '강서구_행정동.json'
gangseo_geo = json.load(open(jpath, encoding='utf-8'))

In [15]:
g_map = folium.Map(location = [37.564284172921866, 126.82663196076031],  
                   tiles='CartoDB positron',
                   zoom_start = 13)
fmap = folium.Choropleth(geo_data = gangseo_geo,
                        data=df_dong,
                        columns=['행정동', '단속건수'],
                        key_on = 'feature.properties.temp', 
                        fill_color='Blues',
                        fill_opacity=0.7,
                        line_opacity=0.3, 
                        legend_name='단속건수', 
                        highlight=True).add_to(g_map)
folium.LayerControl().add_to(g_map)

fmap.geojson.zoom_on_click = False
fmap.geojson.add_child( folium.features.GeoJsonTooltip(['temp'], lables=False))
g_map.save('강서구_단속건수.html')

In [86]:
g_map = folium.Map(location = [37.564284172921866, 126.82663196076031],  
                   tiles='CartoDB positron',
                   zoom_start = 13)
fmap = folium.Choropleth(geo_data = gangseo_geo,
                        data=df_dong,
                        columns=['행정동', '종사자수'],
                        key_on = 'feature.properties.temp', 
                        fill_color='Blues',
                        fill_opacity=0.7,
                        line_opacity=0.3, 
                        legend_name='단속건수', 
                        highlight=True).add_to(g_map)
folium.LayerControl().add_to(g_map)

fmap.geojson.zoom_on_click = False
fmap.geojson.add_child( folium.features.GeoJsonTooltip(['temp'], lables=False))
g_map.save('강서구_종사자수.html')

## 클러스터별 단속건수 군집

In [9]:
import json

jpath = '클러스터_1.json'
gangseo_geo = json.load(open(jpath, encoding='utf-8'))

In [10]:
g_map = folium.Map(location = [37.564284172921866, 126.82663196076031],  
                   tiles='CartoDB positron',
                   zoom_start = 13)
fmap = folium.Choropleth(geo_data = gangseo_geo,
                        data=df_dong,
                        columns=['행정동', '단속건수'],
                        key_on = 'feature.properties.temp', 
                        fill_color='Blues',
                        fill_opacity=0.7,
                        line_opacity=0.3, 
                        legend_name='단속건수', 
                        highlight=True).add_to(g_map)
folium.LayerControl().add_to(g_map)

fmap.geojson.zoom_on_click = False
fmap.geojson.add_child( folium.features.GeoJsonTooltip(['temp'], lables=False))
g_map.save('클러스터1_단속건수.html')

In [11]:
import json

jpath = '클러스터_2.json'
gangseo_geo = json.load(open(jpath, encoding='utf-8'))

In [12]:
g_map = folium.Map(location = [37.564284172921866, 126.82663196076031],  
                   tiles='CartoDB positron',
                   zoom_start = 13)
fmap = folium.Choropleth(geo_data = gangseo_geo,
                        data=df_dong,
                        columns=['행정동', '단속건수'],
                        key_on = 'feature.properties.temp', 
                        fill_color='Blues',
                        fill_opacity=0.7,
                        line_opacity=0.3, 
                        legend_name='단속건수', 
                        highlight=True).add_to(g_map)
folium.LayerControl().add_to(g_map)

fmap.geojson.zoom_on_click = False
fmap.geojson.add_child( folium.features.GeoJsonTooltip(['temp'], lables=False))
g_map.save('클러스터2_단속건수.html')

In [13]:
import json

jpath = '클러스터_0.json'
gangseo_geo = json.load(open(jpath, encoding='utf-8'))

In [14]:
g_map = folium.Map(location = [37.564284172921866, 126.82663196076031],  
                   tiles='CartoDB positron',
                   zoom_start = 13)
fmap = folium.Choropleth(geo_data = gangseo_geo,
                        data=df_dong,
                        columns=['행정동', '단속건수'],
                        key_on = 'feature.properties.temp', 
                        fill_color='Blues',
                        fill_opacity=0.7,
                        line_opacity=0.3, 
                        legend_name='단속건수', 
                        highlight=True).add_to(g_map)
folium.LayerControl().add_to(g_map)

fmap.geojson.zoom_on_click = False
fmap.geojson.add_child( folium.features.GeoJsonTooltip(['temp'], lables=False))
g_map.save('클러스터0_단속건수.html')

In [15]:
import json

jpath = '클러스터_3.json'
gangseo_geo = json.load(open(jpath, encoding='utf-8'))

In [16]:
g_map = folium.Map(location = [37.564284172921866, 126.82663196076031],  
                   tiles='CartoDB positron',
                   zoom_start = 13)
fmap = folium.Choropleth(geo_data = gangseo_geo,
                        data=df_dong,
                        columns=['행정동', '단속건수'],
                        key_on = 'feature.properties.temp', 
                        fill_color='Blues',
                        fill_opacity=0.7,
                        line_opacity=0.3, 
                        legend_name='단속건수', 
                        highlight=True).add_to(g_map)
folium.LayerControl().add_to(g_map)

fmap.geojson.zoom_on_click = False
fmap.geojson.add_child( folium.features.GeoJsonTooltip(['temp'], lables=False))
g_map.save('클러스터3_단속건수.html')

## 클러스터별 주차장 면수 / 종사작수

In [17]:
# 가양2, 가양3, 방화3, 화곡8, 화곡본, 등촌2, 화곡2, 화곡3, 화곡4
# 방화1, 방화2, 염창, 우장산, 등촌1, 등촌3, 화곡6, 화곡1
# 공항, 발산1, 가양1

ratio = []
cluster_0 = df_dong.iloc[[1, 2, 6, 9, 10, 12, 14, 15, 16], :]
cluster_1 = df_dong.iloc[[4, 5, 7, 8, 11, 13, 17, 19], :]
cluster_2 = df_dong.iloc[[0, 3, 18], :]

ratio.append((cluster_0['주차장 면수'] / cluster_0['종사자수']).mean())
ratio.append((cluster_1['주차장 면수'] / cluster_1['종사자수']).mean())
ratio.append((cluster_2['주차장 면수'] / cluster_2['종사자수']).mean())
ratio

[1.935186925442348, 1.3736746532210615, 0.7825470192261338]

In [18]:
df = pd.DataFrame()
df['cluster'] = ['cluster0', 'cluster1', 'cluster2']
df['종사자수 대비 주차장 면수'] = ratio
df

,cluster,종사자수 대비 주차장 면수
0,cluster0,1.935187
1,cluster1,1.373675
2,cluster2,0.782547


In [20]:
import json

jpath = '클러스터별_통합.json'
gangseo_geo = json.load(open(jpath, encoding='utf-8'))

In [21]:
g_map = folium.Map(location = [37.564284172921866, 126.82663196076031],  
                   tiles='CartoDB positron',
                   zoom_start = 13)
fmap = folium.Choropleth(geo_data = gangseo_geo,
                        data=df,
                        columns=['cluster', '종사자수 대비 주차장 면수'],
                        key_on = 'feature.properties.temp', 
                        fill_color='OrRd',
                        fill_opacity=0.7,
                        line_opacity=0.3, 
                        legend_name='종사자수 대비 주차장 면수', 
                        highlight=True).add_to(g_map)
folium.LayerControl().add_to(g_map)

fmap.geojson.zoom_on_click = False
fmap.geojson.add_child( folium.features.GeoJsonTooltip(['temp'], lables=False))
g_map.save('클러스터별_종사자수대비주차장면수.html')

In [28]:
# 가양2, 가양3, 방화3, 화곡8, 화곡본, 등촌2, 화곡2, 화곡3, 화곡4
# 방화1, 방화2, 염창, 우장산, 등촌1, 등촌3, 화곡6, 화곡1
# 공항, 발산1, 가양1

ratio = []
cluster_0 = df_dong.iloc[[1, 2, 6, 9, 10, 12, 14, 15, 16], :]
cluster_1 = df_dong.iloc[[4, 5, 7, 8, 11, 13, 17, 19], :]
cluster_2 = df_dong.iloc[[0, 3, 18], :]

ratio.append(cluster_0['단속건수'].mean())
ratio.append(cluster_1['단속건수'].mean())
ratio.append(cluster_2['단속건수'].mean())
ratio

[4515.333333333333, 6618.125, 26371.0]

In [29]:
df3 = pd.DataFrame()
df3['cluster'] = ['cluster0', 'cluster1', 'cluster2']
df3['단속건수 평균'] = ratio
df3

,cluster,단속건수 평균
0,cluster0,4515.333333
1,cluster1,6618.125000
2,cluster2,26371.000000


In [31]:
import json

jpath = '클러스터별_통합.json'
gangseo_geo = json.load(open(jpath, encoding='utf-8'))

g_map = folium.Map(location = [37.564284172921866, 126.82663196076031],  
                   tiles='CartoDB positron',
                   zoom_start = 13)
fmap = folium.Choropleth(geo_data = gangseo_geo,
                        data=df3,
                        columns=['cluster', '단속건수 평균'],
                        key_on = 'feature.properties.temp', 
                        fill_color='Blues',
                        fill_opacity=0.7,
                        line_opacity=0.3, 
                        legend_name='단속건수 평균', 
                        highlight=True).add_to(g_map)
folium.LayerControl().add_to(g_map)

fmap.geojson.zoom_on_click = False
fmap.geojson.add_child( folium.features.GeoJsonTooltip(['temp'], lables=False))
g_map.save('클러스터별_단속건수 평균.html')

In [164]:
import json

g_map = folium.Map(location = [37.564284172921866, 126.82663196076031],  
                   tiles='CartoDB positron',
                   zoom_start = 13)

jpath = '클러스터_0.json'
gangseo_geo = json.load(open(jpath, encoding='utf-8'))
fmap2 = folium.Choropleth(geo_data = gangseo_geo,
                        data=df_dong,
                        columns=['행정동', '단속건수'],
                        key_on = 'feature.properties.temp', 
                        fill_color='OrRd',
                        fill_opacity=0.7,
                        line_opacity=0.3, 
                        legend_name='단속건수', 
                        highlight=True).add_to(g_map)

jpath = '클러스터_1.json'
gangseo_geo = json.load(open(jpath, encoding='utf-8'))

fmap1 = folium.Choropleth(geo_data = gangseo_geo,
                        data=df_dong,
                        columns=['행정동', '단속건수'],
                        key_on = 'feature.properties.temp', 
                        fill_color='YlGn',
                        fill_opacity=0.7,
                        line_opacity=0.3, 
                        legend_name='단속건수').add_to(g_map)

jpath = '클러스터_2.json'
gangseo_geo = json.load(open(jpath, encoding='utf-8'))
fmap2 = folium.Choropleth(geo_data = gangseo_geo,
                        data=df_dong,
                        columns=['행정동', '단속건수'],
                        key_on = 'feature.properties.temp', 
                        fill_color='RdPu',
                        fill_opacity=0.7,
                        line_opacity=0.3, 
                        legend_name='단속건수', 
                        highlight=True).add_to(g_map)

jpath = '클러스터_3.json'
gangseo_geo = json.load(open(jpath, encoding='utf-8'))
fmap2 = folium.Choropleth(geo_data = gangseo_geo,
                        data=df_dong,
                        columns=['행정동', '단속건수'],
                        key_on = 'feature.properties.temp', 
                        fill_color='Blues',
                        fill_opacity=0.7,
                        line_opacity=0.3, 
                        legend_name='단속건수', 
                        highlight=True).add_to(g_map)

folium.LayerControl(autoZIndex=False).add_to(g_map)

fmap.geojson.zoom_on_click = False
fmap1.geojson.add_child( folium.features.GeoJsonTooltip(['temp'], lables=True))
fmap2.geojson.add_child( folium.features.GeoJsonTooltip(['temp'], lables=True))
g_map.save('test.html')